In [22]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA

# 
from tqdm import tqdm
import sleap

import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage

#
import parmap

import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.database import database
from utils.ethogram import ethogram

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
cohort = database.CohortProcessor()
cohort.fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'
cohort.load_database()

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [17]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################
cohort.n_cores = 30
cohort.parallel = True
cohort.preprocess_feature_tracks()



100%|██████████| 126/126 [00:00<00:00, 1045.03it/s]


In [18]:
#################################################################
###################### PREPROCESS HUDDLES #######################
#################################################################

# 
cohort.parallel = True
cohort.fix_track_flag = True
cohort.interpolate_flag = True
cohort.preprocess_huddle_tracks()



100%|██████████| 126/126 [00:15<00:00,  8.00it/s]


In [95]:
######################################################
############### PROCESS A BEHAVIOR ###################
######################################################

    
#
names = ['female','male','pup1','pup2','pup3','pup4']
a1 = 0
a2 = 1
cohort.behavior_name = ''+names[a1] + " " + names[a2]
cohort.get_pairwise_interaction_time(a1, a2)



100%|██████████| 991/991 [00:03<00:00, 301.22it/s] 

res:  (991,)


In [96]:
###########################################################
###########################################################
###########################################################

#
n_partitions = 24

#
cohort.format_behavior()

#
ethogram.plot_ethogram_hourly(n_partitions, 
                              cohort,
                             vmax=50)




[[15.  9.  0.]
 [15. 10.  0.]
 [15. 10.  0.]
 ...
 [30.  9.  0.]
 [30.  9.  0.]
 [30. 10.  0.]]
start day:  15.0
start time:  9.0
